In [1]:
from bs4 import BeautifulSoup
import os
from glob import glob

import urllib.request
import shutil
from tqdm import tqdm
from pydub import AudioSegment
import multiprocessing as mp
import pickle

## My codes
from src.utils import SoundToText
from src.utils import PrepareSound

## settings
%matplotlib inline

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
!ls Data/NPR

0.mp3	       170.txt	      241_trans.txt  41.mp3	   info_137.pkl
0_trans.txt    171.mp3	      241.txt	     41_trans.txt  info_138.pkl
0.txt	       171_trans.txt  242.mp3	     41.txt	   info_139.pkl
100.mp3        171.txt	      242_trans.txt  42.mp3	   info_13.pkl
100_trans.txt  172.mp3	      242.txt	     42_trans.txt  info_140.pkl
100.txt        172_trans.txt  243.mp3	     42.txt	   info_141.pkl
101.mp3        172.txt	      243_trans.txt  43.mp3	   info_142.pkl
101_trans.txt  173.mp3	      243.txt	     43_trans.txt  info_143.pkl
101.txt        173_trans.txt  244.mp3	     43.txt	   info_144.pkl
102.mp3        173.txt	      244_trans.txt  44.mp3	   info_145.pkl
102_trans.txt  174.mp3	      244.txt	     44_trans.txt  info_146.pkl
102.txt        174_trans.txt  245.mp3	     44.txt	   info_147.pkl
103.mp3        174.txt	      245_trans.txt  45.mp3	   info_148.pkl
103_trans.txt  175.mp3	      245.txt	     45_trans.txt  info_149.pkl
103.txt        175_trans.txt  246.mp3	     45.txt	   info_14.

##  Data source:
##  NPR: https://www.npr.org/podcasts/2038/news-politics

## https://www.npr.org/podcasts/452538677/note-to-self
## https://www.npr.org/podcasts/510298/ted-radio-hour

## https://www.npr.org/series/4516989/storycorps/archive

In [3]:
soup = BeautifulSoup(open("Data/NPR/NPR.html"), "html.parser")

In [4]:
pod_text = []
pod_audi = []

for i in soup.find_all('article', {'class' : 'item podcast-episode'}):
#for i in soup.find_all('article', {'class' : 'item has-image'}):
    #pods.append(i)
    #make sure both audi and text exist for the same episode
    for j in i.find_all('p', {'class': 'teaser'}):
        pod_text.append(j)
    for j in i.find_all('li', {'class': 'audio-tool audio-tool-download'}):
        #link = j.find_all('href')
        pod_audi.append(j)
print(pod_text[0], pod_audi[0])

<p class="teaser">
<time datetime="2018-06-08"><span class="date">June 8, 2018 • </span></time>Before meeting with Kim Jong Un, President Trump heads to Quebec for the G7 summit. He'll meet with world leaders united against his trade policies. Also, a visit to an immigration court in Texas. And why is Pope Francis meeting with oil executives tomorrow?
        </p> <li class="audio-tool audio-tool-download"><a data-metrics='{"category":"audio actions","action":"download audio","label":"618167844","noninteraction":true}' href="https://play.podtrac.com/npr-510318/npr.mc.tritondigital.com/NPR_510318/media/anon.npr-mp3/npr/upfirst/2018/06/20180608_upfirst_upfirst_06082018.mp3?orgId=1&amp;d=784&amp;p=510318&amp;story=618167844&amp;t=podcast&amp;e=618167844&amp;siteplayer=true&amp;dl=1"><b class="icn-download"></b><b class="audio-tool-label">Download</b></a>
</li>


In [5]:
pod_text = []
pod_audi = []

for i in soup.find_all('article', {'class' : 'item podcast-episode'}):
#for i in soup.find_all('article', {'class' : 'item has-image'}):
    #pods.append(i)
    #make sure both audi and text exist for the same episode
    text = i.find_all('p', {'class': 'teaser'})
    audi = i.find_all('li', {'class': 'audio-tool audio-tool-download'})
    if len(text) != 1 or len(audi) != 1:
        ## no download links...
        pass
        #print (len(text), len(audi))
    else:
        pod_text.append(text[0])
        pod_audi.append(audi[0].find_all('a', href=True)[0])
print(pod_text[0], pod_audi[0])

<p class="teaser">
<time datetime="2018-06-08"><span class="date">June 8, 2018 • </span></time>Before meeting with Kim Jong Un, President Trump heads to Quebec for the G7 summit. He'll meet with world leaders united against his trade policies. Also, a visit to an immigration court in Texas. And why is Pope Francis meeting with oil executives tomorrow?
        </p> <a data-metrics='{"category":"audio actions","action":"download audio","label":"618167844","noninteraction":true}' href="https://play.podtrac.com/npr-510318/npr.mc.tritondigital.com/NPR_510318/media/anon.npr-mp3/npr/upfirst/2018/06/20180608_upfirst_upfirst_06082018.mp3?orgId=1&amp;d=784&amp;p=510318&amp;story=618167844&amp;t=podcast&amp;e=618167844&amp;siteplayer=true&amp;dl=1"><b class="icn-download"></b><b class="audio-tool-label">Download</b></a>


In [6]:
print(len(pod_text), len(pod_audi))
N = len(pod_text)

314 314


In [7]:
t = pod_text[0]

In [8]:
## Get teaser
str(t).split("</time>")[1].split("\n")[0]

"Before meeting with Kim Jong Un, President Trump heads to Quebec for the G7 summit. He'll meet with world leaders united against his trade policies. Also, a visit to an immigration court in Texas. And why is Pope Francis meeting with oil executives tomorrow?"

In [9]:
a = pod_audi[0]

In [10]:
a

<a data-metrics='{"category":"audio actions","action":"download audio","label":"618167844","noninteraction":true}' href="https://play.podtrac.com/npr-510318/npr.mc.tritondigital.com/NPR_510318/media/anon.npr-mp3/npr/upfirst/2018/06/20180608_upfirst_upfirst_06082018.mp3?orgId=1&amp;d=784&amp;p=510318&amp;story=618167844&amp;t=podcast&amp;e=618167844&amp;siteplayer=true&amp;dl=1"><b class="icn-download"></b><b class="audio-tool-label">Download</b></a>

In [11]:
## Get download link
a.get("href")

'https://play.podtrac.com/npr-510318/npr.mc.tritondigital.com/NPR_510318/media/anon.npr-mp3/npr/upfirst/2018/06/20180608_upfirst_upfirst_06082018.mp3?orgId=1&d=784&p=510318&story=618167844&t=podcast&e=618167844&siteplayer=true&dl=1'

In [17]:
!ls Data/NPR

NPR.html


In [18]:
path = "Data/NPR/"
for i in tqdm(range(0, len(pod_text))):
    t = pod_text[i]
    ## Get time string
    i_time = t.find("time").get("datetime").replace("-", "")
    ## Get teaser
    i_teaser = str(t).split("</time>")[1].split("\n")[0]
    a = pod_audi[i]
    ## Get download link
    i_url = a.get("href")
    
    with open(path + str(i) + ".txt", "w") as text_file:
        text_file.write(i_teaser)
        text_file.write("\n")
        text_file.write(i_time)
    
    ## download file
    output_file = path + str(i) + ".mp3"
    with urllib.request.urlopen(i_url) as response, open(output_file, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
        
#     sound = AudioSegment.from_mp3(output_file)
    
#     ## split into 20 second pieces
#     for j, chunk in enumerate(sound[::20 * 1000]):
#       with open(path + str(i) + "_" + str(j) + ".wav", "wb") as f:
#         chunk.export(f, format="wav")
    

100%|██████████| 314/314 [20:16<00:00,  3.88s/it]


In [12]:
path = "Data/NPR/"
bigdic = {}
for i in tqdm(range(275, N)):
    bigdic.update(PrepareSound(path + str(i) + ".mp3", silencesplit=True))
    file_path = path ## total 
    inputtasks = glob(file_path + str(i) + "_*.wav")
    #print(" Running %s jobs on %s cores" % (len(inputtasks), mp.cpu_count()-2))
    npool = min(len(inputtasks), mp.cpu_count())
    pool  = mp.Pool(npool)

    fulltextdic = {}
    ## parallel
    for result in pool.map(SoundToText, inputtasks):
            fulltextdic.update(result)
#     ## standard
#     for j in inputtasks:
#         print(j)
#         result = SoundToText(j) #dictionary of values, plots
#         fulltextdic.update(result)
    
    for k in inputtasks:
        os.remove(k)
        
    fulltext = ""
    for l in range(len(fulltextdic.keys())):
        fulltext += fulltextdic[file_path + str(i) + "_" + str(l) + ".wav"] + " ; "
    #print(fulltext)
    with open(file_path + str(i) + "_trans.txt", "w") as text_file:
        text_file.write(fulltext)
    with open(file_path + 'info_' + str(i) + '.pkl', 'wb') as f:
        pickle.dump(fulltextdic, f)
    pool.close()

  3%|▎         | 1/39 [01:09<43:48, 69.17s/it]

cannot recognize  Data/NPR/276_1.wav


  5%|▌         | 2/39 [02:34<47:39, 77.29s/it]

cannot recognize  Data/NPR/277_77.wav
cannot recognize  Data/NPR/277_76.wav
cannot recognize  Data/NPR/277_74.wav


 79%|███████▉  | 31/39 [36:28<09:24, 70.60s/it]

cannot recognize  Data/NPR/306_83.wav
cannot recognize  Data/NPR/306_84.wav


100%|██████████| 39/39 [44:17<00:00, 68.15s/it]


In [19]:
with open(path + 'info.pkl', 'wb') as f:
    pickle.dump(bigdic, f)

NameError: name 'bigdic' is not defined

In [10]:
with open(path + 'info.pkl', 'rb') as f:
    mydic = pickle.load(f)

In [11]:
mydic[path + "0.wav"]

{'dBFS': -21.10909357824653,
 'duration': 802.6122448979592,
 'max': 29039,
 'max_dBFS': -1.0493655646660165,
 'rms': 2884}

## clean the tail files
#### find *.wav -type f -size -500k -delete

In [2]:
#start_time = time.time()
## 228 is missing
for i in tqdm(range(1, 314)): #len(pod_text) 
    file_path = path ## total 
    inputtasks = glob(file_path + str(i) + "_*.wav")
    #print(" Running %s jobs on %s cores" % (len(inputtasks), mp.cpu_count()-2))
    npool = min(len(inputtasks), mp.cpu_count()-1)
    pool  = mp.Pool(npool)

    fulltextdic = {}
    ## parallel
    for result in pool.map(SoundToText, inputtasks):
            fulltextdic.update(result)
#     ## standard
#     for j in inputtasks:
#         print(j)
#         result = SoundToText(j) #dictionary of values, plots
#         fulltextdic.update(result)
    
    fulltext = ""
    for l in range(len(fulltextdic.keys())):
        fulltext += fulltextdic[file_path + str(i) + "_" + str(l) + ".wav"] + " ; "
    #print(fulltext)
    with open(file_path + str(i) + "_trans.txt", "w") as text_file:
        text_file.write(fulltext)
    with open(file_path + 'info_' + str(i) + '.pkl', 'wb') as f:
        pickle.dump(fulltextdic, f)
    pool.close()
#print("--- %s seconds ---" % (time.time() - start_time))

  0%|          | 0/313 [00:00<?, ?it/s]

cannot recognize  Data/NPR/1_1.wav


  1%|          | 2/313 [02:23<6:11:11, 71.61s/it]

cannot recognize  Data/NPR/3_126.wav


  2%|▏         | 5/313 [06:14<6:24:32, 74.91s/it]

...

  4%|▎         | 11/313 [13:21<6:06:37, 72.84s/it]

cannot recognize  Data/NPR/12_1.wav


  4%|▍         | 12/313 [14:29<6:03:28, 72.45s/it]Process ForkPoolWorker-87:
Process ForkPoolWorker-85:
Exception ignored in: <bound method Hypothesis.<lambda> of <pocketsphinx.pocketsphinx.Hypothesis; proxy of <Swig Object of type 'Hypothesis *' at 0x110c09750> >>
Traceback (most recent call last):
  File "/Users/renormalization/anaconda/lib/python3.6/site-packages/pocketsphinx/pocketsphinx.py", line 132, in <lambda>
    __del__ = lambda self: None
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/renormalization/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/renormalization/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/renormalization/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/renormalization/anaconda/lib/python3.6/multiprocessing

KeyboardInterrupt: 

Process ForkPoolWorker-89:
Traceback (most recent call last):
  File "/Users/renormalization/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/renormalization/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/renormalization/anaconda/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/Users/renormalization/anaconda/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/Users/renormalization/Git/Insight/Project/src/utils.py", line 79, in SoundToText
    text = Recon.recognize_sphinx(test_au, language='en-US')
  File "/Users/renormalization/anaconda/lib/python3.6/site-packages/speech_recognition/__init__.py", line 798, in recognize_sphinx
    decoder.process_raw(raw_data, False, True)  # process audio data with recognition enabled (no_search = False), as a 

## Finished!